In [36]:
import httpx
import asyncio
import requests

from tqdm import tqdm

from fetchbim.notion import Database, get_all_pages, Page
from fetchbim.fetch import SharedRule, SharedAttribute
from fetchbim import an_client, NOTION_HEADERS
# from fetchbim.teams import sync_complete_msg

In [37]:
def http_get_sync(url):
    response = requests.get(url, headers=NOTION_HEADERS)
    return response.json()

async def http_get(url):
    return await asyncio.to_thread(http_get_sync, url)

In [44]:
async def get_page_by_id(page_id):
    url = f"https://api.notion.com/v1/pages/{page_id}"
    r = await http_get(url)
    return r

async def main(id_list):
    results = await asyncio.gather(*[get_page_by_id(_id) for _id in id_list])
    pages = [SharedAttribute.from_notion_page(Page(**r)) for r in results]
    return pages

In [27]:
db_rules = SharedRule.get_shared_rules()
print("{} rules in database".format(len(db_rules)))
# Notion to Admin and back
rules = get_all_pages('fd52b1109cf04974b9a76f71a25afc18')
attrs = get_all_pages('b734013d16164030a941e07ff000d0a5')
print("{} rules in Notion".format(len(rules)))

136 rules in database
136 rules in Notion


In [28]:
page_ids = rules[0].properties['SharedAttributes'].get_value()

In [49]:
for page_id in page_ids:
    await get_page_by_id(page_id)

In [48]:
await main(page_ids)

[SharedAttribute(id=9337, name='IFC', value='Furniture', attribute_type=1, sort=0, hidden=False, deleted=False, parameter_type=None, data_type=None),
 SharedAttribute(id=9338, name='BIMobject Category', value='Furniture: Shelving & Storage', attribute_type=1, sort=0, hidden=False, deleted=False, parameter_type=None, data_type=None),
 SharedAttribute(id=9339, name='Tags', value='MityBilt, MIB, 342338', attribute_type=1, sort=0, hidden=False, deleted=False, parameter_type=None, data_type=None),
 SharedAttribute(id=9340, name='Primary Material', value='High pressure laminate', attribute_type=1, sort=0, hidden=False, deleted=False, parameter_type=None, data_type=None),
 SharedAttribute(id=9341, name='Assembly Code', value='E20', attribute_type=0, sort=0, hidden=False, deleted=False, parameter_type='Type', data_type='Text'),
 SharedAttribute(id=9342, name='Manufacturer', value='Southwest Solutions Group_MIB', attribute_type=0, sort=0, hidden=False, deleted=False, parameter_type='Type', data

In [4]:
sr = SharedRule.from_notion_page(rules[0])

ValidationError: 6 validation errors for SharedRule
Attributes -> 0
  value is not a valid dict (type=type_error.dict)
Attributes -> 1
  value is not a valid dict (type=type_error.dict)
Attributes -> 2
  value is not a valid dict (type=type_error.dict)
Attributes -> 3
  value is not a valid dict (type=type_error.dict)
Attributes -> 4
  value is not a valid dict (type=type_error.dict)
Attributes -> 5
  value is not a valid dict (type=type_error.dict)

In [5]:
sr

NameError: name 'sr' is not defined

In [3]:
for rule in tqdm(rules):
    # notion to python object
    shared_rule = SharedRule.from_notion(rule)
    # object to DB
    shared_rule.post()

100%|██████████| 120/120 [06:52<00:00,  3.44s/it]


In [4]:
# # Clean slate in notion
SharedFile.archive_notion('Shared Rules')
SharedFile.archive_notion('Shared Attributes')

In [2]:
table_dict = {}
count = 0
# DB data to notion
db_rules = SharedFile.get_all()
shared_rules_ids = [x['SharedFileId'] for x in db_rules]
for id_ in tqdm(shared_rules_ids):
    sf = SharedFile.from_json(id_)
    sf.to_notion()
    count += 1

table_dict['Total Shared Rules'] = count
sync_complete_msg(
    title="Shared Rule Sync Complete",
    message="Two way sync between Notion 'Shared Rules' database and admin 'Shared Files' successfully completed",
    button=("Shared Rules", "https://www.notion.so/fetchbim/fd52b1109cf04974b9a76f71a25afc18"),
    table_dict=table_dict
)

100%|██████████| 120/120 [16:31<00:00,  8.26s/it]
